In [1]:
from ultralytics import YOLO
import cv2
# Load a model
model = YOLO('models/yolov8x-pose.onnx', verbose=False)  # load an official model
# model = YOLO('path/to/best.pt')  # load a custom model

image = cv2.imread(r"frames\1_avi\frame_0.jpg")
results = model(image)  # predict on an image

# for r in results:
#     print(r.keypoints.xy)  # print string of prediction results

Loading models\yolov8x-pose.onnx for ONNX Runtime inference...

0: 640x640 (no detections), 755.6ms
Speed: 11.4ms preprocess, 755.6ms inference, 687.3ms postprocess per image at shape (1, 3, 640, 640)


In [2]:
import math

def get_angle(point1, point2, point3):
    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    
    angle1 = math.atan2(y2 - y1, x2 - x1)
    angle2 = math.atan2(y3 - y2, x3 - x2)
    
    angle = math.degrees(angle2 - angle1)
    # angle = abs(angle)
    return angle

# Example usage
point1 = (0, 0)
point2 = (3, 3)
point3 = (6, 0)
angle = get_angle(point1, point2, point3)
print(angle)

-90.0


In [3]:
video_path = 'dataset/escalator-fall/1.avi'
# model = YOLO('models/yolov8x-pose.pt', verbose=False)  # load an official model
results = model(video_path, stream=True)  # predict on an image

In [3]:
import cv2
from ultralytics import YOLO
import pandas as pd
import numpy as np

# Load the YOLOv8 model
# model = YOLO('yolov8x-pose.pt')
video_path = 'dataset/escalator-fall/1.avi'
# Open the video file
cap = cv2.VideoCapture(video_path)
frame_count = 0
threshold_angle = 50
df = pd.DataFrame(columns=["person_number", "frame_number", "side", "angle"])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        if frame_count % 1 == 0:  # Skip 10 frames
            neccessary_points = [12, 14, 16, 11, 13, 15]
            results = model(frame, stream=False)  # predict on an image
            print(frame_count)

            # Loop through all keypoints
            r=results[0]
            x_y_points = []
            for person_number in range(len(r.keypoints[0].xy)):
                if len(r.keypoints[0].xy[person_number]) != 17:
                    print('keypoint empty')
                    continue
                # Get the x and y coordinates of the keypoint
                for point_number in neccessary_points:
                    x = int(r.keypoints[0].xy[person_number][point_number][0])
                    y = int(r.keypoints[0].xy[person_number][point_number][1])
                    if x == 0:
                        print(f'{frame_count}, empty, x:{x}, y:{y}')
                        x_y_points = []
                        break
                    x_y_points.append((x, y))
                    # Draw a circle at the keypoint location with the random color
                    # cv2.circle(image2, (int(x), int(y)), 5, color, -1)
                if len(x_y_points) == 0:
                    continue
                # print(x_y_points)
                right_angle = get_angle(x_y_points[0], x_y_points[1], x_y_points[2])
                left_angle = get_angle(x_y_points[3], x_y_points[4], x_y_points[5])

                if right_angle > threshold_angle and left_angle > threshold_angle:
                    print(f'{frame_count}, BENT ? {right_angle}, {left_angle}')
                    df = pd.concat([df, pd.DataFrame({'person_number': person_number, 'frame_number': frame_count, 'side': ['right', 'left'], 'angle': [right_angle, left_angle]})], ignore_index=True)
                    continue
                # Append side and angle to df dataframe

    else:
        # Break the loop if the end of the video is reached
        break
    
    # Read the next frame
    frame_count += 1

# Release the video capture object and close the display window
cap.release()
# df.to_csv('df1.csv')


0: 640x640 (no detections), 700.3ms
Speed: 3.0ms preprocess, 700.3ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)
0
keypoint empty

0: 640x640 (no detections), 733.5ms
Speed: 7.0ms preprocess, 733.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
1
keypoint empty

0: 640x640 (no detections), 718.4ms
Speed: 4.5ms preprocess, 718.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
2
keypoint empty

0: 640x640 (no detections), 719.4ms
Speed: 5.5ms preprocess, 719.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
3
keypoint empty

0: 640x640 (no detections), 729.6ms
Speed: 3.0ms preprocess, 729.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
4
keypoint empty

0: 640x640 (no detections), 724.5ms
Speed: 4.0ms preprocess, 724.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
5
keypoint empty

0: 640x640 (no detections), 719.5ms
Speed: 5.0ms preprocess, 719.5ms inference, 1.0ms p

KeyboardInterrupt: 

In [5]:
import pandas as pd

df = pd.read_csv('1_avi.csv', index_col=0)
df_filtered = df[(abs(df['left_angle']) > 60) & (abs(df['right_angle']) > 60)]

df_filtered.index = range(len(df_filtered))
df_filtered

,person_number,frame_number,left_angle,right_angle
0,0,110,-65.480357,-76.087505
1,0,111,-71.565051,-93.945186
2,0,114,-72.814294,-64.440035
3,0,115,-96.340192,-75.564025
4,0,116,-100.619655,-90.842524
...,...,...,...,...
120,0,1344,-127.203009,-101.364604
121,0,1345,-118.383354,-96.051732
122,0,1346,-102.858090,-87.735245
123,0,1347,-105.851928,-70.377744


In [ ]:
df = pd.read_csv('1_false_avi.csv', index_col=0)
df_filtered = df[(abs(df['left_angle']) > 60) & (abs(df['right_angle']) > 60)]

df_filtered